In [ ]:
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
import requests
import json

In [ ]:
#file path
#Import clean csv file
project_df_path = "Data/merge.csv"
project_df = pd.read_csv(project_df_path)
project_df.head()

In [ ]:
#see dataframe column as a list
#project_df.columns.tolist()

In [ ]:
#number of participating countries
project_df["Country or region_HS"].unique()

In [ ]:
#suppress warning
import warnings
warnings.simplefilter("ignore")

In [ ]:
#Get ranking and totals data
totals_df = project_df[['Year_HS', 'Country_SP', 'Overall rank_HS', 'Score_HS', 'SPI Rank_SP',  'Social Progress Index_SP']]

#Rename year and country columns
totals_df.rename(columns = {"Year_HS":"Year", "Country_SP":"Country"}, inplace = True)

#configure SPI ranking to not have a decimal
totals_df["SPI Rank_SP"] = totals_df["SPI Rank_SP"].astype(int)

#renaming columns to get rid of spaces
totals_df.rename(columns={"Overall rank_HS":"Rank_HS"}, inplace=True)
totals_df.rename(columns={"SPI Rank_SP":"Rank_SPI"}, inplace=True)

In [ ]:
#Top Rankings of Happiness Score
#ranking_HS = totals_df.sort_values(["Year","Rank_HS"])
#ranking_HS

In [ ]:
#Top Rankings of Social Progress Index
#ranking_SP = totals_df.sort_values(["Year","Rank_SPI"])
#ranking_SP

In [ ]:
ranking_both_top10 = totals_df[(totals_df["Rank_HS"] <= 10) & (totals_df["Rank_SPI"] <= 10)]
#use \ (vertical bar for either condiation)
ranking_both_top10

In [ ]:
ranking_HS = ranking_both_top10.sort_values(["Rank_HS"])

In [ ]:
ranking_SPI = ranking_both_top10.sort_values(["Rank_SPI"])
#ranking_SPI

In [ ]:
ranking_country = ranking_both_top10.sort_values(["Country"])
ranking_country

In [ ]:
ranking_top10 = ranking_both_top10.groupby(["Country","Rank_HS","Rank_SPI"])
ranking_top10.count()

In [ ]:
count10 = ranking_top10["Rank_SPI"].count()
count10

In [ ]:
#Scatterplot all years 
x_values = ranking_both_top10["Rank_HS"]
y_values = ranking_both_top10["Rank_SPI"]

In [ ]:
plt.title("Top 10 Social Progress and Happiness Rankings (2015-2019)")
plt.ylabel(" Social Progress Rank")
plt.xlabel("Happiness Rank")
plt.ylim(0,11)
plt.xlim(0,11)
plt.grid(True, linestyle="-", which="major", color="gray", alpha=0.25)

plt.scatter(x_values, y_values, color="black", marker="o")
plt.show()


In [ ]:
#plot_country = ranking_country["Country"].to_list()
#plot_HS_rank = ranking_country["Rank_HS"].to_list()
#plot_SPI_rank = ranking_country["Rank_SPI"].to_list()
#plot_HS_rank
#plot_SPI_rank
#plot_country

In [ ]:
#Create variable for list of ranking data
plot_HS_rank = ranking_country["Rank_HS"].to_list()

plot_SPI_rank = ranking_country["Rank_SPI"].to_list()

country = ranking_country["Country"].to_list()


In [ ]:
#Create variable to add and inverse the rankings so the smallest number for the ranking displays a larger dot using list comprehension.
#https://www.geeksforgeeks.org/python-adding-two-list-elements/
#display_dot = [plot_HS_rank[i] + plot_SPI_rank[i] for i in range(len(plot_HS_rank))]

display_dot = [float(plot_HS_rank[i]) + float(plot_SPI_rank[i]) for i in range(len(plot_HS_rank))]

#Find reciprocal
#https://www.geeksforgeeks.org/numpy-reciprocal
display_dot_inv = np.reciprocal(display_dot)

#display_dot
#display_dot_inv

In [ ]:
#Scatterplot of the two rankings with cities that rank 1-10 in both.
#https://stackoverflow.com/questions/26139423/plot-different-color-for-different-categorical-levels-using-matplotlib
#https://python-graph-gallery.com/270-basic-bubble-plot/

df = pd.DataFrame(dict(plot_HS_rank=plot_HS_rank, plot_SPI_rank=plot_SPI_rank, country=country, display_dot_inv=display_dot_inv))
fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (7,7)

colors = {'Australia':'crimson', 'Canada':'darkviolet', 'Denmark':'green', 'Finland':'deepskyblue', 'Netherlands':'gold', 'New Zealand':'orange', 'Norway':'mediumblue', 'Sweden':'teal', 'Switzerland':'yellowgreen'}
scatter = ax.scatter(df['plot_HS_rank'], df['plot_SPI_rank'], s=df['display_dot_inv']*2500, c=df['country'].apply(lambda x: colors[x]))
legend1 = ax.legend(*scatter.legend_elements(), loc="lower right", title="Countries")

plt.ylim(-1,11)
plt.xlim(-1,11)
plt.title("Countries that Rank in the Top 10 in Both \n Social Progress and Happiness Rankings (2015-2019)", fontsize=15)
plt.ylabel(" Social Progress Rank", fontsize=12)
plt.xlabel("Happiness Rank", fontsize=12)
plt.grid(True, linestyle="-", which="major", color="gray", alpha=0.25)

#plt.savefig("Data/TopTenRanking.png")
plt.show()

In [ ]:
groups = ranking_country.groupby("Country")
colors = {'Australia':'crimson', 'Canada':'darkviolet', 'Denmark':'green', 'Finland':'deepskyblue', 'Netherlands':'gold', 'New Zealand':'orange', 'Norway':'mediumblue', 'Sweden':'teal', 'Switzerland':'yellowgreen'}

for name, group in groups:
    plt.plot(group["Rank_HS"], group["Rank_SPI"], marker="o", markersize=20, linestyle="", label=name,)

plt.rcParams["figure.figsize"] = (7,7)
plt.rcParams["legend.markerscale"] = 0.4

plt.ylim(-1,11)
plt.xlim(-1,11)
plt.title("Countries that Rank in the Top 10 in Both \n Social Progress and Happiness Rankings (2015-2019)", fontsize=15)
plt.ylabel(" Social Progress Rank", fontsize=12)
plt.xlabel("Happiness Rank", fontsize=12)
plt.grid(True, linestyle="-", which="major", color="gray", alpha=0.25)
plt.legend()

#plt.savefig("Data/TopTenRanking-alternate.png")
plt.show()

In [ ]:
#Scatterplot all years 
x_values = totals_df["Score_HS"]
y_values = totals_df["Social Progress Index_SP"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = slope * x_values + intercept
line_eq = "y =" + str(round(slope,2)) + "x " + str(round(intercept,2))

plt.rcParams["figure.figsize"] = (7,7)

plt.title("Social Progress Index and Happiness Scores \n for All Countries (2015-2019)",  fontsize=15)
plt.ylabel(" Social Progress Index",  fontsize=12)
plt.xlabel("Happiness Score",  fontsize=12)
plt.ylim(20,100)
plt.xlim(0,10)
plt.grid(True, linestyle="-", which="major", color="gray", alpha=0.25)
plt.annotate(line_eq, (2,30), fontsize=12, color="red")

plt.scatter(x_values, y_values, color="black", marker="o")
plt.plot(x_values, regress_values, "r-")

#plt.savefig("Data/HP_SPI_scatter.png")

plt.show()

#print r squared
print(f"R squared: {rvalue**2}")

In [ ]:
#Group by country
grouped_totals_df = totals_df.groupby(["Country"])
grouped_totals_df.mean()

In [ ]:
#Converting groupby to a dataframe
averages_df = pd.DataFrame(grouped_totals_df["Rank_HS", "Score_HS", "Rank_SPI", "Social Progress Index_SP"].mean())

In [ ]:
#Scatterplot averaged data 
x_values = averages_df["Score_HS"]
y_values = averages_df["Social Progress Index_SP"]

plt.rcParams["figure.figsize"] = (7,7)

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = slope * x_values + intercept
line_eq = "y =" + str(round(slope,2)) + "x " + str(round(intercept,2))

plt.title("5-year Average Social Progress Index \n and Happiness Score for Each Country",  fontsize=15)
plt.ylabel(" Social Progress Index",  fontsize=12)
plt.xlabel("Happiness Score",  fontsize=12)
plt.ylim(20,100)
plt.xlim(0,10)
plt.grid(True, linestyle="-", which="major", color="gray", alpha=0.25)
plt.annotate(line_eq, (2,30), fontsize=12, color="red")

plt.scatter(x_values, y_values, color="black", marker="o")
plt.plot(x_values, regress_values, "r-")

#plt.savefig("Data/HP_SPI_5average_scatter.png")

plt.show()

#print r squared
print(f"R squared: {rvalue**2}")

# Analysis of Relationships between Social Progress Indicators and Happiness Score

### Basic Human Needs vs. Happiness Linear Regression

In [ ]:
# Add the linear regression equation and line to plot
x_values = project_df['Score_HS']
y_values = project_df['Basic Human Needs_SP']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values, edgecolors= "black")
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(5,45),fontsize=15,color="red")

# Labels
plt.title('Basic Human Needs vs. Happiness Linear Regression')
plt.xlabel('Hapiness Score')
plt.ylabel('Basic Human Needs Score')
plt.grid()
plt.show()

In [ ]:
score = f"The r value is: {rvalue}"
if rvalue == 0:
    relationship = "none existant"
    strength = ""
else:
    if rvalue > 0:
        relationship = "positive"
    else:
        relationship = "negative"

if abs(rvalue) >= .7:
    strength = "strong"
elif abs(rvalue) >= .5:
    strength = "moderate"
elif abs(rvalue) >= .3 and rvalue != 0:
    strength = "weak"
    
print(f"The r value is: {rvalue}.  This is a {strength} {relationship} relationship")

### Foundations of Wellbeing vs Happiness Linear Regression

In [ ]:
# Add the linear regression equation and line to plot
x_values = project_df['Score_HS']
y_values = project_df['Foundations of Wellbeing_SP']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values, edgecolors= "black")
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(5,51),fontsize=15,color="red")

# Labels
plt.title('Foundations of Wellbeing  vs. Happiness Linear Regression')
plt.xlabel('Hapiness Score')
plt.ylabel('Foundations of Wellbeing Score')
plt.grid()
plt.show()

In [ ]:
score = f"The r value is: {rvalue}"
if rvalue == 0:
    relationship = "none existant"
    strength = ""
else:
    if rvalue > 0:
        relationship = "positive"
    else:
        relationship = "negative"

if abs(rvalue) >= .7:
    strength = "strong"
elif abs(rvalue) >= .5:
    strength = "moderate"
elif abs(rvalue) >= .3 and rvalue != 0:
    strength = "weak"
    
print(f"The r value is: {rvalue}.  This is a {strength} {relationship} relationship")

### Opportunity vs. Happiness Linear Regression

In [ ]:
# Add the linear regression equation and line to plot
x_values = project_df['Score_HS']
y_values = project_df['Opportunity_SP']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values, edgecolors= "black")
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(5.7,40),fontsize=15,color="red")

# Labels
plt.title('Opportunity  vs. Happiness Linear Regression')
plt.xlabel('Hapiness Score')
plt.ylabel('Foundations of Wellbeing Score')
plt.grid()
plt.show()

In [ ]:
score = f"The r value is: {rvalue}"
if rvalue == 0:
    relationship = "none existant"
    strength = ""
else:
    if rvalue > 0:
        relationship = "positive"
    else:
        relationship = "negative"

if abs(rvalue) >= .7:
    strength = "strong"
elif abs(rvalue) >= .5:
    strength = "moderate"
elif abs(rvalue) >= .3 and rvalue != 0:
    strength = "weak"
    
print(f"The r value is: {rvalue}.  This is a {strength} {relationship} relationship")

# Finding Sub-metric with Strongest Correlation to Happiness Score

In [ ]:
# Create lists for storage of information
sub_metric_list = []
rvalue_list = []

# Select only columns relating to Social Progress Indicators
submetric_df = project_df.iloc[:, 22:84]

# Iterate through the submetric data frame and store rvalues and submetric label for each
for column in submetric_df:
    submetric_string = column[:-3]
    x_values = project_df['Score_HS']
    y_values = project_df[column]
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    sub_metric_list.append(submetric_string)
    rvalue_list.append(rvalue)
    
# Store lists in a dictionary
submetric_results_dict = {
    'Sub Metric' : sub_metric_list,
    'R Value' : rvalue_list
}

# Create a dataframe with the dictionary
submetric_results_df = pd.DataFrame(data = submetric_results_dict)

### Positive Relationship Results

In [ ]:
positive_results_df = submetric_results_df.loc[submetric_results_df['R Value'] >= 0]
weak_pos_results_df = positive_results_df.sort_values(by=['R Value'])
strong_pos_results_df = positive_results_df.sort_values(by=['R Value'], ascending = False)

<b>Strongest Positive Indicators of Happiness</b>

In [ ]:
strong_pos_results_df.head()

<b>Weakest Positive Indicators of Happiness</b>

In [ ]:
weak_pos_results_df.head()

### Negative Relationship Results

In [ ]:
negative_results_df = submetric_results_df.loc[submetric_results_df['R Value'] < 0]
strong_neg_results_df = negative_results_df.sort_values(by=['R Value'])
weak_neg_results_df = negative_results_df.sort_values(by=['R Value'], ascending = False)

<b>Strongest Negative Indicators of Happiness</b>

In [ ]:
strong_neg_results_df.head()

<b>Weakest Negative Indicators of Happiness</b>

In [ ]:
weak_neg_results_df.head()

# Analysis of Relationships between Happiness Indicators and Social Progress Score

In [ ]:
# Add the linear regression equation and line to plot
x_values = project_df['GDP per capita_HS']
y_values = project_df['Social Progress Index_SP']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values, edgecolors= "black")
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,( 1.1, 60),fontsize=15,color="red")

# Labels
plt.title('GDP per capita vs. Social Progress Index Score')
plt.xlabel('GDP per capita')
plt.ylabel('Social Progress Index Score')
plt.grid()
plt.scatter(x_values, y_values, color="black", marker="o")
plt.show()

In [ ]:
score = f"The r value is: {rvalue}"
if rvalue == 0:
    relationship = "none existant"
    strength = ""
else:
    if rvalue > 0:
        relationship = "positive"
    else:
        relationship = "negative"

if abs(rvalue) >= .7:
    strength = "strong"
elif abs(rvalue) >= .5:
    strength = "moderate"
elif abs(rvalue) >= .3 and rvalue != 0:
    strength = "weak"
    
print(f"The r value is: {rvalue}.  This is a {strength} {relationship} relationship")

In [ ]:
x_values = project_df['GDP per capita_HS']
y_values = project_df['Social Progress Index_SP']

correlation_matrix = np.corrcoef(x_values, y_values)
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2

#print(r_squared)
print (f"The R^2 value for GDP per Capita is: {r_squared}. GDP per capita is the highest happiness indicator that predicts social progress score." )

In [ ]:
# Add the linear regression equation and line to plot
x_values = project_df['Social support_HS']
y_values = project_df['Social Progress Index_SP']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values, edgecolors= "black")
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,( 1.1, 60),fontsize=15,color="red")

# Labels
plt.title('Social support vs. Social Progress Index Score')
plt.xlabel('Social support')
plt.ylabel('Social Progress Index Score')
plt.grid()
plt.scatter(x_values, y_values, color="black", marker="o")
plt.show()

There is a strong positive relationship between GDP per capita and Social Progress Index score, therefore as the GDP of the countries increase the social progress index will also increase and vice versa.

In [ ]:
score = f"The r value is: {rvalue}"
if rvalue == 0:
    relationship = "none existant"
    strength = ""
else:
    if rvalue > 0:
        relationship = "positive"
    else:
        relationship = "negative"

if abs(rvalue) >= .7:
    strength = "strong"
elif abs(rvalue) >= .5:
    strength = "moderate"
elif abs(rvalue) >= .3 and rvalue != 0:
    strength = "weak"
    
print(f"The r value is: {rvalue}.  This is a {strength} {relationship} relationship")

In [ ]:
x_values = project_df['Social support_HS']
y_values = project_df['Social Progress Index_SP']

correlation_matrix = np.corrcoef(x_values, y_values)
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2

#print(r_squared)
print (f"The R^2 value for Social support is: {r_squared}. Social support is the third highest happiness indicator that predicts Social Progress score" )

In [ ]:
# Add the linear regression equation and line to plot
x_values = project_df['Healthy life expectancy_HS']
y_values = project_df['Social Progress Index_SP']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values, edgecolors= "black")
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,( 0.7, 60),fontsize=15,color="red")

# Labels
plt.title('Healthy life expectancy vs. Social Progress Index Score')
plt.xlabel('Healthy life expectancy')
plt.ylabel('Social Progress Index Score')
plt.grid()
plt.scatter(x_values, y_values, color="black", marker="o")
plt.show()

There is a moderate positive relationship between Social support of countries and Social Progress index score, therefore whenever the social support increase the social progress index score of the respective countries increases moderately. And vice versa.

In [ ]:
score = f"The r value is: {rvalue}"
if rvalue == 0:
    relationship = "none existant"
    strength = ""
else:
    if rvalue > 0:
        relationship = "positive"
    else:
        relationship = "negative"

if abs(rvalue) >= .7:
    strength = "strong"
elif abs(rvalue) >= .5:
    strength = "moderate"
elif abs(rvalue) >= .3 and rvalue != 0:
    strength = "weak"
    
print(f"The r value is: {rvalue}.  This is a {strength} {relationship} relationship")


In [ ]:
x_values = project_df['Healthy life expectancy_HS']
y_values = project_df['Social Progress Index_SP']

correlation_matrix = np.corrcoef(x_values, y_values)
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2

#print(r_squared)
print (f"The R^2 value for Healthy life expectancy is: {r_squared}. Healthy life expectancy is the second highest happiness indicator that predicts social progress score." )


There is a strong positive relationship between the Healthy life expectancy of countries and the social progress index score, thus when the healthy life expectancy of a specific country increases the social progress index score of the respective country increases as well.

In [ ]:
# Add the linear regression equation and line to plot
x_values = project_df['Freedom to make life choices_HS']
y_values = project_df['Social Progress Index_SP']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values, edgecolors= "black")
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,( 0.5, 60),fontsize=15,color="red")

# Labels
plt.title('Freedom to make life choices vs. Social Progress Index Score')
plt.xlabel('Freedom to make life choices')
plt.ylabel('Social Progress Index Score')
plt.grid()
plt.scatter(x_values, y_values, color="black", marker="o")
plt.show()

In [ ]:
score = f"The r value is: {rvalue}"
if rvalue == 0:
    relationship = "none existant"
    strength = ""
else:
    if rvalue > 0:
        relationship = "positive"
    else:
        relationship = "negative"

if abs(rvalue) >= .7:
    strength = "strong"
elif abs(rvalue) >= .5:
    strength = "moderate"
elif abs(rvalue) >= .2 and rvalue != 0:
    strength = "weak"
    
print(f"The r value is: {rvalue}.  This is a {strength} {relationship} relationship")

There is a weak positive correlation between Freedom to make life choices and Social progress Index Score, therefore freedom to make life choices does not affect Social Progress Index.

In [ ]:
# Add the linear regression equation and line to plot
x_values = project_df['Generosity_HS']
y_values = project_df['Social Progress Index_SP']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values, edgecolors= "black")
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,( 0.5, 60),fontsize=15,color="red")

# Labels
plt.title('Generosity vs. Social Progress Index Score')
plt.xlabel('Generosity')
plt.ylabel('Social Progress Index Score')
plt.grid()
plt.scatter(x_values, y_values, color="black", marker="o")
plt.show()

In [ ]:
score = f"The r value is: {rvalue}"
if rvalue == 0:
    relationship = "none existant"
    strength = ""
else:
    if rvalue > 0:
        relationship = "positive"
    else:
        relationship = "negative"

if abs(rvalue) >= .7:
    strength = "strong"
elif abs(rvalue) >= .5:
    strength = "moderate"
elif abs(rvalue) >= .1 and rvalue != 0:
    strength = "weak"
    
print(f"The r value is: {rvalue}.  This is a {strength} {relationship} relationship")

There is a weak positive relationship between Generosity and Social Progress Index Score, therefore generosity of countries will not affect the social progress index score of the respective countries.

In [ ]:
# Add the linear regression equation and line to plot
x_values = project_df['Perceptions of corruption_HS']
y_values = project_df['Social Progress Index_SP']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values, edgecolors= "black")
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,( 0.5, 60),fontsize=15,color="red")

# Labels
plt.title('Perceptions of corruption vs. Social Progress Index Score')
plt.xlabel('Perceptions of corruption')
plt.ylabel('Social Progress Index Score')
plt.grid()
plt.scatter(x_values, y_values, color="black", marker="o")
plt.show()

In [ ]:
score = f"The r value is: {rvalue}"
if rvalue == 0:
    relationship = "none existant"
    strength = ""
else:
    if rvalue > 0:
        relationship = "positive"
    else:
        relationship = "negative"

if abs(rvalue) >= .7:
    strength = "strong"
elif abs(rvalue) >= .5:
    strength = "moderate"
elif abs(rvalue) >= .1 and rvalue != 0:
    strength = "weak"
    
print(f"The r value is: {rvalue}.  This is a {strength} {relationship} relationship")

There is also a weak positive relationship between Perceptions of Corruption and Social Progress Score, therefore we can conclude and say that Social Progress Index of Countries will not be affected by Perceptions of Corruption.